In [1]:
# 1. 데이터 및 환경설정 (T5 기반 Dialogue Summarization)

import os
import re
import json
import yaml
from glob import glob
from pprint import pprint

import pandas as pd
from tqdm import tqdm

import torch
from torch.utils.data import Dataset, DataLoader

import pytorch_lightning as pl  # 쓰지 않더라도 requirements 맞추기용
from rouge import Rouge

from transformers import (
    AutoTokenizer,
    T5ForConditionalGeneration,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    EarlyStoppingCallback,
)

import wandb  # 선택: 학습 로깅용

# =========================================
# 1-1. 기본 설정: T5 한국어 요약 모델 선택
# =========================================
MODEL_NAME = "eenzeenee/t5-base-korean-summarization"  # T5-base 한국어 요약 모델

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# T5 계열은 보통 bos_token이 없으므로 None 방지
bos_token = tokenizer.bos_token if tokenizer.bos_token is not None else ""  # decoder input 앞에 붙일 토큰 (없어도 됨)
eos_token = tokenizer.eos_token if tokenizer.eos_token is not None else ""  # 보통 "</s>"

# =========================================
# 1-2. YAML Config 구성
# =========================================
config_data = {
    "general": {
        "data_path": "./data/",           # train.csv, dev.csv, test.csv 위치
        "model_name": MODEL_NAME,          # 사용할 T5 모델
        "output_dir": "./t5_outputs",      # 체크포인트 저장 디렉토리
    },
    "tokenizer": {
        "encoder_max_len": 512,            # 대화가 꽤 길 수 있어서 512 유지
        "decoder_max_len": 80,             # 요약은 상대적으로 짧게
        "bos_token": bos_token,
        "eos_token": eos_token,
        # 대화 내 특수 토큰을 보존하기 위해 special_tokens 등록
        "special_tokens": [
            "#Person1#", "#Person2#", "#Person3#",
            "#Person4#", "#Person5#", "#Person6#", "#Person7#",
            "#PhoneNumber#", "#Address#", "#PassportNumber#"
        ],
        # T5 요약 프롬프트
        "t5_prefix": "summarize: "
    },
    "training": {
        "overwrite_output_dir": True,
        "num_train_epochs": 10,            # KoBART 20ep → T5 8~12ep 권장, 일단 10
        "learning_rate": 5e-5,             # 요약 태스크용으로 자주 쓰는 lr
        "per_device_train_batch_size": 8,  # OOM 방지 & 안정적인 학습
        "per_device_eval_batch_size": 16,
        "warmup_ratio": 0.1,
        "weight_decay": 0.01,
        "lr_scheduler_type": "cosine",
        "optim": "adamw_torch",
        "gradient_accumulation_steps": 1,
        "evaluation_strategy": "epoch",
        "save_strategy": "epoch",
        "save_total_limit": 5,
        "fp16": True,
        "load_best_model_at_end": True,
        "seed": 42,
        "logging_dir": "./logs",
        "logging_strategy": "epoch",
        "predict_with_generate": True,     # ROUGE 계산을 위해 generate 사용
        "generation_max_length": 80,
        "metric_for_best_model": "rouge-l",  # ROUGE-L 기준으로 best ckpt 선택
        "greater_is_better": True,
        "do_train": True,
        "do_eval": True,
        "early_stopping_patience": 3,
        "early_stopping_threshold": 0.001,
        "report_to": "none",             # "wandb" 또는 "none"
    },
    "wandb": {
        "entity": "quriquri7",    # 예: "quriquri7"
        "project": "dialogue-summ-t5",     # 원하는 프로젝트명
        "name": "t5-base-run1"            # run 이름
    },
    "inference": {
        "ckt_path": "추론에_사용할_체크포인트_경로_또는_output_dir",  # 예: "t5_outputs/checkpoint-xxxx"
        "result_path": "./prediction_t5/",
        "no_repeat_ngram_size": 3,        # 반복 감소 → ROUGE 상승에 도움
        "early_stopping": True,
        "generate_max_length": 80,
        "num_beams": 5,                   # 빔 서치; 4~8 범위 튜닝 추천
        "batch_size": 32,
        "remove_tokens": [
            "<usr>",
            bos_token,
            eos_token,
            tokenizer.pad_token if tokenizer.pad_token is not None else "",
        ],
    },
}

# =========================================
# 1-3. Config 저장 및 로드
# =========================================
os.makedirs("./", exist_ok=True)
config_path = "./config_t5.yaml"

with open(config_path, "w", encoding="utf-8") as f:
    yaml.dump(config_data, f, allow_unicode=True)

with open(config_path, "r", encoding="utf-8") as f:
    loaded_config = yaml.safe_load(f)

print("=== Loaded Config ===")
pprint(loaded_config["general"])
pprint(loaded_config["tokenizer"])
pprint(loaded_config["training"])
pprint(loaded_config["wandb"])
pprint(loaded_config["inference"])

# =========================================
# 1-4. 데이터 로드 확인
# =========================================
data_path = loaded_config["general"]["data_path"]

train_df = pd.read_csv(os.path.join(data_path, "train.csv"))
val_df   = pd.read_csv(os.path.join(data_path, "dev.csv"))

print("\n=== Train Sample ===")
print(train_df.tail())
print("\n=== Dev Sample ===")
print(val_df.tail())


/data/ephemeral/home/nlp_server/venv310/lib/python3.10/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


=== Loaded Config ===
{'data_path': './data/',
 'model_name': 'eenzeenee/t5-base-korean-summarization',
 'output_dir': './t5_outputs'}
{'bos_token': '',
 'decoder_max_len': 80,
 'encoder_max_len': 512,
 'eos_token': '</s>',
 'special_tokens': ['#Person1#',
                    '#Person2#',
                    '#Person3#',
                    '#Person4#',
                    '#Person5#',
                    '#Person6#',
                    '#Person7#',
                    '#PhoneNumber#',
                    '#Address#',
                    '#PassportNumber#'],
 't5_prefix': 'summarize: '}
{'do_eval': True,
 'do_train': True,
 'early_stopping_patience': 3,
 'early_stopping_threshold': 0.001,
 'evaluation_strategy': 'epoch',
 'fp16': True,
 'generation_max_length': 80,
 'gradient_accumulation_steps': 1,
 'greater_is_better': True,
 'learning_rate': 5e-05,
 'load_best_model_at_end': True,
 'logging_dir': './logs',
 'logging_strategy': 'epoch',
 'lr_scheduler_type': 'cosine',
 'metric_for_b

In [2]:
# 2. 데이터 가공 및 Dataset 클래스 (T5 정석 패턴)

from torch.utils.data import Dataset

class DialogTrainDataset(Dataset):
    def __init__(self, df, tokenizer, encoder_max_len, decoder_max_len, prefix="summarize: "):
        self.tokenizer = tokenizer
        self.encoder_max_len = encoder_max_len
        self.decoder_max_len = decoder_max_len
        self.prefix = prefix

        self.dialogues = df["dialogue"].tolist()
        self.summaries = df["summary"].tolist()

        # 미리 토큰화 해두기 (메모리 여유 있으니 괜찮음)
        encodings = tokenizer(
            [self.prefix + d for d in self.dialogues],
            max_length=self.encoder_max_len,
            truncation=True,
            padding="max_length",
            return_tensors="pt",
        )
        # label용 summary 토큰화
        with tokenizer.as_target_tokenizer():
            labels = tokenizer(
                self.summaries,
                max_length=self.decoder_max_len,
                truncation=True,
                padding="max_length",
                return_tensors="pt",
            )["input_ids"]

        # pad → -100 (loss에서 무시)
        labels[labels == tokenizer.pad_token_id] = -100

        self.input_ids = encodings["input_ids"]
        self.attention_mask = encodings["attention_mask"]
        self.labels = labels

    def __len__(self):
        return len(self.dialogues)

    def __getitem__(self, idx):
        return {
            "input_ids": self.input_ids[idx],
            "attention_mask": self.attention_mask[idx],
            "labels": self.labels[idx],
        }


class DialogTestDataset(Dataset):
    def __init__(self, df, tokenizer, encoder_max_len, prefix="summarize: "):
        self.tokenizer = tokenizer
        self.encoder_max_len = encoder_max_len
        self.prefix = prefix

        self.fnames = df["fname"].tolist()
        self.dialogues = df["dialogue"].tolist()

        encodings = tokenizer(
            [self.prefix + d for d in self.dialogues],
            max_length=self.encoder_max_len,
            truncation=True,
            padding="max_length",
            return_tensors="pt",
        )

        self.input_ids = encodings["input_ids"]
        self.attention_mask = encodings["attention_mask"]

    def __len__(self):
        return len(self.dialogues)

    def __getitem__(self, idx):
        return {
            "input_ids": self.input_ids[idx],
            "attention_mask": self.attention_mask[idx],
            "fname": self.fnames[idx],
        }


# train / val Dataset 생성
encoder_max_len = loaded_config["tokenizer"]["encoder_max_len"]
decoder_max_len = loaded_config["tokenizer"]["decoder_max_len"]
t5_prefix = loaded_config["tokenizer"]["t5_prefix"]

train_dataset = DialogTrainDataset(
    train_df,
    tokenizer,
    encoder_max_len=encoder_max_len,
    decoder_max_len=decoder_max_len,
    prefix=t5_prefix,
)

val_dataset = DialogTrainDataset(
    val_df,
    tokenizer,
    encoder_max_len=encoder_max_len,
    decoder_max_len=decoder_max_len,
    prefix=t5_prefix,
)

len(train_dataset), len(val_dataset)


/data/ephemeral/home/nlp_server/venv310/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


(12457, 499)

In [3]:
# 3. Trainer 및 TrainingArguments (T5 + DataCollator, wandb 없음)

from transformers import (
    T5ForConditionalGeneration,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
)
from rouge import Rouge
import numpy as np

def compute_metrics_t5(pred):
    """
    Trainer가 generate한 요약 vs gold summary(labels)를 ROUGE로 평가
    """
    rouge = Rouge()

    preds = pred.predictions
    labels = pred.label_ids

    # -100 → pad_token_id로 복구해서 디코딩
    preds[preds == -100] = tokenizer.pad_token_id
    labels[labels == -100] = tokenizer.pad_token_id

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # 디버깅용 일부 출력
    print("-" * 150)
    for i in range(3):
        if i >= len(decoded_preds):
            break
        print(f"[PRED {i}]: {decoded_preds[i][:150]}")
        print(f"[GOLD {i}]: {decoded_labels[i][:150]}")
        print("-" * 150)

    # ROUGE 계산
    scores = rouge.get_scores(decoded_preds, decoded_labels, avg=True)
    result = {k: v["f"] for k, v in scores.items()}  # F1만 사용

    # Trainer 로그용으로 numpy 변환
    return {k: float(v) for k, v in result.items()}


def load_t5_model(config, device):
    model_name = config["general"]["model_name"]
    model = T5ForConditionalGeneration.from_pretrained(model_name)

    # special_tokens 추가 (발화자/개인정보 토큰)
    special_tokens_dict = {"additional_special_tokens": config["tokenizer"]["special_tokens"]}
    tokenizer.add_special_tokens(special_tokens_dict)
    model.resize_token_embeddings(len(tokenizer))

    model.to(device)
    return model


from transformers import set_seed
set_seed(loaded_config["training"]["seed"])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = load_t5_model(loaded_config, device)

# DataCollator - padding & labels 정리 자동 처리
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
)

training_args = Seq2SeqTrainingArguments(
    output_dir=loaded_config["general"]["output_dir"],
    overwrite_output_dir=True,
    num_train_epochs=5,              # 일단 5epoch 정도로 시작
    learning_rate=3e-5,              # 조금 낮게
    per_device_train_batch_size=4,   # 메모리 여유에 따라 조정
    per_device_eval_batch_size=8,
    warmup_ratio=0.1,
    weight_decay=0.01,
    lr_scheduler_type="cosine",
    optim="adamw_torch",
    gradient_accumulation_steps=1,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=3,
    fp16=True,
    load_best_model_at_end=True,
    seed=loaded_config["training"]["seed"],
    logging_dir=loaded_config["training"]["logging_dir"],
    logging_strategy="epoch",
    predict_with_generate=True,
    generation_max_length=loaded_config["training"]["generation_max_length"],
    generation_num_beams=loaded_config["inference"]["num_beams"],
    metric_for_best_model="rouge-l",
    greater_is_better=True,
    do_train=True,
    do_eval=True,
    report_to="none",               # 🔥 wandb 완전 비활성화
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics_t5,
)


/data/ephemeral/home/nlp_server/venv310/lib/python3.10/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/data/ephemeral/home/nlp_server/venv310/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [4]:
# 4. 모델 학습하기 (T5 정석 파이프라인)

train_result = trainer.train()
print("✅ 학습 완료")

# best model 로드되어 있음 (load_best_model_at_end=True)

# dev에서 샘플 하나 직접 요약해보기
sample = val_df.iloc[0]
text = loaded_config["tokenizer"]["t5_prefix"] + sample["dialogue"]

inputs = tokenizer(
    text,
    return_tensors="pt",
    truncation=True,
    max_length=loaded_config["tokenizer"]["encoder_max_len"],
).to(device)

with torch.no_grad():
    output_ids = model.generate(
        **inputs,
        max_length=loaded_config["training"]["generation_max_length"],
        num_beams=loaded_config["inference"]["num_beams"],
        no_repeat_ngram_size=3,
    )

gen = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print("\n[DEV 샘플 요약 테스트]")
print("GEN :", gen)
print("GOLD:", sample["summary"])


You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge-1,Rouge-2,Rouge-l
1,1.089900,0.860992,0.271599,0.103334,0.260741
2,0.849400,0.822655,0.274060,0.101674,0.259277
3,0.761300,0.817566,0.283521,0.104598,0.268993
4,0.706600,0.816271,0.290288,0.109769,0.275376
5,0.681600,0.820255,0.289509,0.109579,0.275468


------------------------------------------------------------------------------------------------------------------------------------------------------
[PRED 0]: #Person1#은 #Person2#에게 천식 검사를 위해 폐 전문의에게 가보라고 조언합니다.
[GOLD 0]: #Person2#는 숨쉬기 어려워합니다. 의사는 #Person2#에게 증상을 확인하고, 천식 검사를 위해 폐 전문의에게 가볼 것을 권합니다.
------------------------------------------------------------------------------------------------------------------------------------------------------
[PRED 1]: #Person1#과 Jimmy는 3시 30분에 체육관에서 운동을 하기로 결정합니다.
[GOLD 1]: #Person1#는 Jimmy를 운동하러 초대하고 팔과 복근 운동을 하도록 설득합니다.
------------------------------------------------------------------------------------------------------------------------------------------------------
[PRED 2]: #Person2#는 #Person1#에게 건강에 안 좋은 음식을 그만 먹으라고 조언합니다.
[GOLD 2]: #Person1#은 건강에 안 좋은 음식을 그만 먹기로 결심하고, #Person2#는 자신의 건강한 식단을 #Person1#에게 공유합니다.
---------------------------------------------------------------------------------------------------------------------------------

There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


✅ 학습 완료

[DEV 샘플 요약 테스트]
GEN : 양 씨는 숨쉬기가 어렵고 가슴이 답답한 증상을 호소합니다. 두 사람은 천식 검사를 위해 폐 전문의에게 가보라고 제안합니다.
GOLD: #Person2#는 숨쉬기 어려워합니다. 의사는 #Person2#에게 증상을 확인하고, 천식 검사를 위해 폐 전문의에게 가볼 것을 권합니다.


In [5]:
# 5. 모델 추론하기 (T5 + DialogTestDataset 버전)

import os
from torch.utils.data import DataLoader
from transformers import T5ForConditionalGeneration, AutoTokenizer
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("-" * 10, f"device : {device}", "-" * 10)

# 5-1. 체크포인트 경로 설정 (이미 학습한 best ckpt로 교체)
loaded_config["inference"]["ckt_path"] = "t5_outputs/checkpoint-12460"  # 실제 best ckpt 경로
ckpt_path = loaded_config["inference"]["ckt_path"]

# 5-2. 토크나이저 & 모델 로드 (훈련 때와 동일하게 special_tokens 추가)
model_name = loaded_config["general"]["model_name"]
tokenizer = AutoTokenizer.from_pretrained(model_name)

special_tokens_dict = {"additional_special_tokens": loaded_config["tokenizer"]["special_tokens"]}
tokenizer.add_special_tokens(special_tokens_dict)

model = T5ForConditionalGeneration.from_pretrained(ckpt_path)
model.resize_token_embeddings(len(tokenizer))
model.to(device)
model.eval()

print("-" * 10, "Inference model & tokenizer loaded", "-" * 10)

# 5-3. test.csv 로드 & DialogTestDataset 생성
test_df = pd.read_csv(os.path.join(loaded_config["general"]["data_path"], "test.csv"))

encoder_max_len = loaded_config["tokenizer"]["encoder_max_len"]
t5_prefix = loaded_config["tokenizer"]["t5_prefix"]

# 👉 2번 셀에서 정의한 DialogTestDataset 를 그대로 사용
test_dataset = DialogTestDataset(
    test_df,
    tokenizer,
    encoder_max_len=encoder_max_len,
    prefix=t5_prefix,
)

test_loader = DataLoader(
    test_dataset,
    batch_size=loaded_config["inference"]["batch_size"],
)

print(f"test 데이터 개수: {len(test_dataset)}")

# 5-4. generate로 요약 생성
all_fnames = []
all_summaries = []

no_repeat_ngram_size = loaded_config["inference"]["no_repeat_ngram_size"]
early_stopping = loaded_config["inference"]["early_stopping"]
max_length = loaded_config["inference"]["generate_max_length"]
num_beams = loaded_config["inference"]["num_beams"]

with torch.no_grad():
    for batch in tqdm(test_loader):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        fnames = batch["fname"]

        outputs = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_length=max_length,
            num_beams=num_beams,
            no_repeat_ngram_size=no_repeat_ngram_size,
            early_stopping=early_stopping,
        )

        decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)

        all_fnames.extend(fnames)
        all_summaries.extend(decoded)

# 5-5. 필요하면 노이즈 토큰 제거 (지금은 skip_special_tokens=True라 크게 필요 없긴 함)
# remove_tokens = loaded_config["inference"]["remove_tokens"]
# cleaned_summaries = all_summaries.copy()
# for token in remove_tokens:
#     if token:
#         cleaned_summaries = [s.replace(token, " ") for s in cleaned_summaries]

# 5-6. 제출용 CSV 저장
output = pd.DataFrame(
    {
        "fname": all_fnames,
        "summary": all_summaries,  # 필요하면 위에서 cleaned_summaries로 교체
    }
)

result_path = loaded_config["inference"]["result_path"]
os.makedirs(result_path, exist_ok=True)

save_path = os.path.join(result_path, "output_t5.csv")
output.to_csv(save_path, index=False, encoding="utf-8-sig")

print(f"✅ Inference 완료, 파일 저장: {save_path}")
output.head()


---------- device : cuda ----------


/data/ephemeral/home/nlp_server/venv310/lib/python3.10/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


---------- Inference model & tokenizer loaded ----------
test 데이터 개수: 499


100%|██████████| 16/16 [01:13<00:00,  4.61s/it]

✅ Inference 완료, 파일 저장: ./prediction_t5/output_t5.csv


,fname,summary
0,test_0,양 씨는 Ms. Dawson에게 모든 직원들에게 사내 메모를 보내달라고 요청합니다....
1,test_1,양 씨는 교통체증으로 인해 집에 갈 때 다른 길을 찾는 것을 제안하고 있습니다. 두...
2,test_2,Kate는 Masha와 Hero가 두 달 동안 별거 끝에 이혼 신청을 했다고 말한다...
3,test_3,Brian은 Anna의 생일을 축하하며 파티에서 함께 춤을 추자고 제안한다.
4,test_4,양 씨와 차 씨는 올림픽 공원의 규모에 대해 이야기하고 있다.
